# Processing Raw Mobile Positioning Data (MPD)

## Summary

This notebook is used to process raw MPD. The process involves using `PySpark` to process raw mobile phone data from CSV/Parquet files. The data is read into Spark dataframes and transformed by rounding the `cell location data` and using reverse geocoding to obtain administrative location information. 

The data is preprocessed by `removing duplicates`, `anomaly subscribers`, and `remove random records`. Finally, the processed data is saved as a new CSV/Parquet file for further analysis. The goal of this process is to clean and transform the raw mobile phone data into a more usable format for analysis, allowing for valuable insights to be extracted from the data.

In general, MPD data is processed `regularly` every month, so it is assumed that the input data provided is separate for each month. This timeframe can be `readjusted` according to work objectives and the availability of resources to process data.

## Requirements

- PySpark installed on the local machine / PySpark Cluster with HDFS 

- Required packages and dependencies installed (pyspark, pandas, geopandas, folium)

- Raw mobile phone data (MPD) in CSV / Parquet Format.

**To run this notebook, you must install spark locally or having access to spark cluster, and install all packages dependency.** 

Please take a look into the [Environment Notebook: Mac & Linux Users](https://github.com/mandes95/893SSA-2022-BDT-DKH/blob/main/notebook/00-Setup%20Environment%20%5Bmac%20%26%20linux%5D.ipynb) or [Environment Notebook: Windows Users](https://github.com/mandes95/893SSA-2022-BDT-DKH/blob/main/notebook/00-Setup%20Environment%20%5Bwindows%5D.ipynb)

## Data Structure

To work with Mobile Positioning Data (MPD), the minimum required fields are listed below:

### Option 1: Records/Events Data Already Merged with Cells Location Data

| Field Name   | Type      | Mode     | Description                                          |
|--------------|-----------|----------|------------------------------------------------------|
| `msisdn`     | String    |          | Hashed subscribers identifier                        |
| `datetime`   | Timestamp |          | Transaction date (date and hour)                     |
| `cell_id`    | String    | NULLABLE | Hashed cell identifier                               |
| `latitude`   | Float     |          | Latitude of Base Transceiver Station (BTS)           |
| `longitude`  | Float     |          | Longitude of Base Transceiver Station (BTS)          |
| `data_type`  | String    |          | Data source, can be CDR/CHG or IPDR/UPCC             |
| `service`    | String    |          | Transaction service (4G/ 3G/ 2G)                     |

### Option 2: Records Data Not Merged with Cells Location Data

#### Records data

| Field Name   | Type      | Mode     | Description                                          |
|--------------|-----------|----------|------------------------------------------------------|
| `msisdn`     | String    |          | Hashed subscribers identifier                        |
| `datetime`   | Timestamp |          | Transaction date (date and hour)                     |
| `cell_id`    | String    |          | Hashed cell identifier                               |
| `data_type`  | String    |          | Data source, can be CDR/CHG or IPDR/UPCC             |

#### Cells Data

| Field Name   | Type      | Mode     | Description                                          |
|--------------|-----------|----------|------------------------------------------------------|
| `cell_id`    | String    |          | Hashed cell identifier                               |
| `latitude`   | Float     |          | Latitude of Base Transceiver Station (BTS)           |
| `longitude`  | Float     |          | Longitude of Base Transceiver Station (BTS)          |
| `service`    | String    |          | Transaction service (4G/ 3G/ 2G)                     |

To successfully run the processing of raw MPD using this notebook, please ensure to follow the same structure or adjust the script as needed. 

Moreover, additional information from the subscribers demography data will be nice to have for getting more informative information especially related to the gender and age group. 

**Note**: Please ensure the hashed cell ID is `consistent` with the records/events dataset so that it can be merged.

### Demography of Subscribers (additional)

Additionally, we can also add the user demography data (nice to have) to be merged with the records data so that it can be used for more detail analysis based on the gender or age_group of the subscribers.

| Field name   | Type   | Mode     | Description                                             |
|--------------|--------|----------|---------------------------------------------------------|
| `msisdn`     | String |          | Hashed subscribers identifier                           |
| `age`        | Int    |          | Subscribers age from registration data                  |
| `gender`     | String |          | Subscribers gender (M/F) from registration data         |

<br>

## Processing Steps

1. Initialize `Spark Session`, then read the `raw MPD` dataset using PySpark.

2. `Transform` the cell location by rounding the latitude and longitude values to a certain decimal place (4 decimal places, equal to accuracy ~11.1 m. [Ref.](http://wiki.gis.com/wiki/index.php/Decimal_degrees)). Will be useful if the cell ID information is missing.

3. `Exploratory data analysis (EDA)` that involves investigating and summarizing the main characteristics of raw MPD, such as the information about how many unique subscribers, records distribution, etc.

4. Preprocess the data by `removing duplicates`, `anomaly subscribers` (e.g. machine, tourist), and `fast movers/random records`.

5. `Save` the processed data as a new CSV or Parquet file for further analysis.

In [ ]:
# Import the sys module to adjust the Python system path
import sys

# Import the os module to interact with the operating system
import os

# Install folium inside the *same environment* as the active kernel
import subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", "folium"])

# Append a path to the system path so that Python can import modules from the specified directory (relative path)
sys.path.append('../')

# Import the CONF, QA_summary and QA_action_plan variables from the script.QA module
from script.conf import *

In [ ]:
try:
    flag = os.path.exists(BASE_PATH+RAW_FILE_PATH)
    if flag:
        USE_MERGED = True
    else:
        USE_MERGED = False
        print("Merged file not found. Will proceed the data using option unmerged. Please update configuration of RAW_CELL_PATH and RAW_SUBS_PATH")
except:
    USE_MERGED = False
    print("Merged file not found. Will proceed the data using option unmerged. Please update configuration of RAW_CELL_PATH and RAW_SUBS_PATH")

print("Use Merged: {}".format(USE_MERGED))

## Start PySpark

In [ ]:
# Import the PySpark module
import pyspark
from pyspark.sql.types import StructType, StructField, StringType, FloatType, TimestampType

import pyspark.sql.functions as f
from pyspark.sql.functions import year, month, dayofmonth, substring, col, to_date
from pyspark.sql.window import Window

# Import the SparkSession from PySpark
from pyspark.sql import SparkSession

# if got error due to port configuration, run this script on terminal
# sudo hostname -s 127.0.0.1

# Create a SparkSession with the specified configuration local
spark = SparkSession.builder\
        .master("local[{}]".format(CORE))\
        .appName("01.ITU.PySpark-Raw")\
        .config('spark.sql.execution.arrow.pyspark.enabled', 'true')\
        .config('spark.eventLog.gcMetrics.youngGenerationGarbageCollectors', 'true')\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# Print the spark object which contains the SparkSession
spark

In [ ]:
# Use this code below to connect your remote spark cluster instead.

# # Import the PySpark module
# import pyspark
# from pyspark.sql.types import StructType, StructField, StringType, FloatType, TimestampType

# import pyspark.sql.functions as f
# from pyspark.sql.functions import year, month, dayofmonth, substring, col, to_date
# from pyspark.sql.window import Window

# # Import the SparkSession from PySpark
# from pyspark.sql import SparkSession

# CLUSTER_URL = "spark://master"
# PORT = "7077"

# # Create a SparkSession with the specified configuration remote server
# spark = SparkSession.builder\
#         .master("{}:{}".format(CLUSTER_URL,PORT))\
#         .appName("01.ITU.PySpark-Raw")\
#         .getOrCreate()

# # Print the spark object which contains the SparkSession
# spark

## Read Raw MPD
The following code code sets a `BASE_PATH` variable to define the parent directory where data files are located. Then, it reads a specific CSV file into a `DataFrame` object using Apache Spark's `read()` method. 

If the available data is already in combined form (**option 1**), in this example refer to the CSV file `"MPD_sample.csv"`, which will be loaded from `BASE_PATH`. 

If the available data is still separated between records and cells (**option 2**), in this example refer to the CSV file `"MPD_sample_records.csv"` and `"MPD_sample_cells.csv"`, which will also be loaded from `BASE_PATH`. 

The resulting `DataFrame` is printed to display the first five rows in a tabular format, and then the total count of records in the `DataFrame` is printed using the `count()` method.

### Set MPD Schema

To import data using PySpark, its better to set specific classes by defines a schema for a set of mandatory columns in a data file using the `StructType()` class.

For This notebook, the schema is composed of seven mandatory MPD fields, each defined as a `StructField()` object that specifies the name, data type, and whether the field can contain null values. The available data types are specified by the imported classes (i.e., `StringType()`, `FloatType()`, and `TimestampType()`).

Once the `StructType()` object is created, it is assigned to the variable `MPDSchema_mandatory`, allowing it to be used elsewhere in the code.

### Option 1: Merged Data

In [ ]:
# if USE_MERGED:
#   # Define a schema for a mandatory set of columns in a data file
#   # Use StructType to create an object that defines the structure of the schema
#   MPD_schema_mandatory = StructType([ \
#       StructField("msisdn", StringType(), True), \
#       StructField("datetime", TimestampType(), True), \
#       StructField("cell_id", StringType(), True), \
#       StructField("latitude", FloatType(), True), \
#       StructField("longitude", FloatType(), True), \
#       StructField("data_type", StringType(), True), \
#       StructField("service", StringType(), True) \
#     ])

#   # Read in a CSV file using Apache Spark's DataFrame API
#   # Set options including delimiter and header presence
#   # Provide a schema using MPDSchema_mandatory variable defined elsewhere
#   # Load the specified CSV file into a DataFrame
#   df = spark.read.format('csv')\
#       .options(delimiter=',')\
#       .option('header',True)\
#       .schema(MPD_schema_mandatory)\
#       .load(BASE_PATH+RAW_FILE_PATH)

#   # Display the first five rows of the DataFrame in a tabular format
#   df.show(5)

#   # Print the number of records in the DataFrame
#   print("Number of records: {}".format(df.count()))

In [ ]:
from pyspark.sql.functions import coalesce, trim, regexp_replace, try_to_timestamp, lit, col

if USE_MERGED:

    from pyspark.sql.types import StructType, StructField, StringType, FloatType
    from pyspark.sql.functions import col, trim, regexp_replace, to_timestamp, coalesce, date_format

    # 1) Schema: read datetime as STRING first (so multiple patterns can be tried)
    MPD_schema_mandatory = StructType([
        StructField("msisdn",    StringType(), True),
        StructField("datetime",  StringType(), True),   # <- StringType here
        StructField("cell_id",   StringType(), True),
        StructField("latitude",  FloatType(),   True),
        StructField("longitude", FloatType(),   True),
        StructField("data_type", StringType(),  True),
        StructField("service",   StringType(),  True),
    ])

    df = (spark.read.format('csv')
          .options(delimiter=',')
          .option('header', True)
          .schema(MPD_schema_mandatory)
          .load(BASE_PATH + RAW_FILE_PATH))

    # 2) Normalize whitespace in the datetime text
    dt_clean = trim(regexp_replace(col("datetime"), r"\s+", " "))

    # 3) Try multiple patterns (Java SimpleDateFormat, not Python!):
    #    - 12h with/without seconds + AM/PM
    #    - 24h with/without seconds
    #    - common ISO-like variants (optional)
    patterns = [
        "M/d/yyyy h:mm:ss a",   # e.g., 9/22/2025 6:05:00 AM
        "M/d/yyyy h:mm a",      # e.g., 9/22/2025 6:05 AM
        "M/d/yyyy H:mm:ss",     # e.g., 9/22/2025 11:34:10
        "M/d/yyyy H:mm",        # e.g., 9/22/2025 11:34
        "MM/dd/yyyy HH:mm:ss",  # zero-padded month/day (optional)
        "yyyy-MM-dd HH:mm:ss",  # common ISO without 'T'
        "yyyy-MM-dd'T'HH:mm:ss'Z'",      # ISO UTC with Z
        "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'",  # ISO UTC with millis
    ]

    #parsed = coalesce(*[to_timestamp(dt_clean, p) for p in patterns])
    parsed = coalesce(*[try_to_timestamp(dt_clean, lit(p)) for p in patterns])

    # 4) Overwrite the datetime column as proper TimestampType
    df = df.withColumn("datetime", parsed)

    # Inspect
    df.show(5)
    print("Number of records: {}".format(df.count()))
    print("Unparsed rows:", df.filter(col("datetime").isNull()).count())  

### Option 2: Separated Data

In [ ]:
if USE_MERGED==False:
  # Define a schema for a mandatory set of columns in a data file
  # Use StructType to create an object that defines the structure of the schema

  # records data schema
  MPD_record_schema_mandatory = StructType([ \
      StructField("msisdn", StringType(), True), \
      StructField("datetime", TimestampType(), True), \
      StructField("cell_id", StringType(), True), \
      StructField("data_type", StringType(), True), \
    ])

  # cells data schema
  MPD_cell_schema_mandatory = StructType([ \
      StructField("cell_id", StringType(), True), \
      StructField("latitude", FloatType(), True), \
      StructField("longitude", FloatType(), True), \
      StructField("service", StringType(), True) \
    ])

  # Read in a CSV file using Apache Spark's DataFrame API
  # Set options including delimiter and header presence
  # Load the records data using records schema
  df_records = spark.read.format('csv')\
      .options(delimiter=',')\
      .option('header',True)\
      .schema(MPD_record_schema_mandatory)\
      .load(BASE_PATH+RAW_FILE_PATH)

  # Load the cells data using cells schema
  df_cells = spark.read.format('csv')\
      .options(delimiter=',')\
      .option('header',True)\
      .schema(MPD_cell_schema_mandatory)\
      .load(BASE_PATH+RAW_CELLS_PATH)

  # Joining two dataframes using left join on cell_id
  df = df_records.join(df_cells,df_records.cell_id ==  df_cells.cell_id,"left").drop(df_cells.cell_id)

  # Reorder columns
  df = df.select('msisdn','datetime','cell_id','latitude','longitude','data_type','service')

  # Display the first five rows of the DataFrame in a tabular format
  df.show(5)

  # Print the number of records in the DataFrame
  print("Number of records: {}".format(df.count()))

In [ ]:
# display column names and types
df.printSchema()

### Rounding Decimal And Create New Column

To start working with raw MPD, we need to rounding the decimal into standard precision (4 decimals, equal to 11.1 m) and create new column for identifying the date.

The code takes the DataFrame `df` and uses the `.withColumn()` method to add three new columns to it. The first two columns are 'latitude' and 'longitude', and they are created by rounding the existing 'latitude' and 'longitude' columns to 4 decimal places using the `f.round()` method from the PySpark SQL functions library (`f`), and then assigning these rounded values to the new columns.

The third column is 'date', which is created by converting the existing 'datetime' column to a date type using the `to_date()` function from the `pyspark.sql.functions` module, and then assigning the converted values to the new column.

In [ ]:
# Round the 'latitude' column of the DataFrame `df` to 4 decimal places and create a new column called 'latitude' with the rounded values.
# Do the same for the 'longitude' column and create a new column called 'longitude' with the rounded values.
# Then convert the 'datetime' column to a date type and create a new column called 'date' with the converted values.

df = df\
    .withColumn(
        'latitude',f.round(col('latitude'),4) # round the 'latitude' column
    )\
    .withColumn(
        'longitude',f.round(col('longitude'),4) # round the 'longitude' column
    )\
    .withColumn(
        'date',to_date(col('datetime')) # convert the 'datetime' column to a date type
    )

# Show the first 5 rows of the updated DataFrame `df`.
df.show(5)


### Drop Duplicate Rows

This code utilizes the `dropDuplicates()` method to efficiently eliminate duplicate rows from the data frame. 
By invoking this method, all duplicate rows are eradicated, leaving only a single instance of each unique row. If there are no changes observed before and after applying this operation, it signifies the absence of any duplicates in the data frame.

In [ ]:
# detect duplicate rows
df_duplicates = df.groupBy(df.columns).count().filter("count > 1")
print(f"number of duplicate rows: {df_duplicates.count()}")
df_duplicates.show()

In [ ]:
# Print the number of records in the DataFrame
print("Number of records before deduplication: {}\n".format(df.count()))

# Drops the duplicate rows from the dataframe
df = df.dropDuplicates()

# Display the first five rows of the DataFrame in a tabular format
df.show(5)

# Print the number of records in the DataFrame
print("Number of records after deduplication: {}".format(df.count()))

### Caching Data in PySpark

Caching data is a technique used in PySpark to improve the performance of iterative and interactive operations on large datasets. When you cache a DataFrame or RDD (Resilient Distributed Dataset), PySpark stores the data in memory and allows subsequent operations to access the data more quickly, avoiding the need to recompute or fetch the data from disk.

`Caching` a dataframe improves the `performance` of subsequent operations that use this dataframe as it avoids reading the data from disk on every operation. However, caching requires **extra memory** usage and should be used judiciously to avoid memory overflow issues.

When caching a DataFrame or RDD in PySpark, it's essential to understand the `active period` of the cache. The active period refers to the duration during which the data remains cached in memory and readily available for quick access.

By default, PySpark caches data in memory using a storage level called `MEMORY_AND_DISK`. This storage level ensures that the data is stored in memory as long as possible, but if the memory becomes insufficient, it spills the excess data to disk to avoid out-of-memory errors.

The active period of the cache starts when you explicitly cache the DataFrame or RDD using the cache() method. From that point onward, the cached data remains available for efficient retrieval by subsequent operations until one of the following events occurs:

- **Explicit Unpersist**: If you invoke the `unpersist()` method on the DataFrame or RDD, the cached data is immediately removed from memory, and the active period ends. 

- **Spark Context Termination**: When the Spark application or Spark Context terminates, either by explicitly stopping the context or when the application finishes execution, the active period of the cache ends. At this point, all cached data is automatically unpersisted, and the memory is released.

In [ ]:
# The following code caches the dataframe in memory or on disk, if there's enough memory to hold all partitions of the dataframe.
df.cache()

### Exploratory Data Analysis (EDA)

`Exploratory Data Analysis (EDA)` is a crucial process in data science that involves investigating and summarizing the `primary characteristics` of a dataset. The main objective of EDA is to gain insights into the raw data and comprehend its underlying structure, identify patterns, detect anomalies, and develop hypotheses for further analysis.

When processing mobile positioning data, EDA involves examining the dataset to understand the available variables and their distributions, such as the number of unique subscribers, unique cell locations, and the distribution of records. This process can also help to identify any missing or erroneous values and diagnose general patterns within the data.

EDA is an essential step in preparing mobile positioning data for subsequent analysis as it identifies potential issues that require addressing, including data cleaning or imputation, and so on.

#### Checking Number of Unique Subscribers, Cell ID, Data type, & Services

The following operations are performed:
1. Four columns ('msisdn', 'cell_id', 'data_type', and 'service') are selected from the dataframe `df`, which we want to analyze.
2. `countDistinct()` function is applied on each of these columns to get the count of unique entities present in them.
3. The obtained values are then renamed, using `alias()` method, to better represent their count metrics.
4. `show()` method is used to display the resulting table in the console.

In [ ]:
# Selects four columns from the dataframe df and counts their distinct values to get insights about unique subscribers, cell ids, data types and services.
df.select(
      f.countDistinct(col('msisdn')).alias('unique_subscribers'), # Counting distinct values of 'msisdn' column and giving it an alias name 'unique_subscribers'
      f.countDistinct(col('cell_id')).alias('unique_cell_id'), # Counting distinct values of 'cell_id' column and giving it an alias name 'unique_cell_id'
      f.countDistinct(col('data_type')).alias('unique_data_type'), # Counting distinct values of 'data_type' column and giving it an alias name 'unique_data_type'
      f.countDistinct(col('service')).alias('unique_service') # Counting distinct values of 'service' column and giving it an alias name 'unique_service'
  ).show() # Displaying the resulting dataframe using show() method

#### Checking the range of datetime, latitude, & Longitude

This code generates the range values for datetime, latitude, and longitude columns in a given dataframe. 

It does this by performing the following operations:
1. Selecting specific columns from the data frame using `select()`, with each selected value having an alias to rename it.
2. Using PySpark's built-in SQL functions `min()` and `max()` along with PySpark's `col()` function to select the minimum and maximum values of datetime, latitude, and longitude respectively in the data frame.
3. Renaming each of these selected minimum and maximum values as per their column name with the `alias()` method.
4. Finally, displaying all the selected and renamed values using the `show()` method.

In [ ]:
# Generate the range value of datetime, latitude, and longitude
df_range = df\
    .select(
        # Selects the minimum value of 'datetime' column and renames it to 'min_datetime'
        f.min(col('datetime')).alias('min_datetime'),
        # Selects the maximum value of 'datetime' column and renames it to 'max_datetime'
        f.max(col('datetime')).alias('max_datetime'),
        # Selects the minimum value of 'latitude' column and renames it to 'min_latitude'
        f.min(col('latitude')).alias('min_latitude'),
        # Selects the minimum value of 'longitude' column and renames it to 'min_longitude'
        f.min(col('longitude')).alias('min_longitude'),
        # Selects the maximum value of 'latitude' column and renames it to 'max_latitude'
        f.max(col('latitude')).alias('max_latitude'),
        # Selects the maximum value of 'longitude' column and renames it to 'max_longitude'
        f.max(col('longitude')).alias('max_longitude'),
    )

df_range.show() 

In [ ]:
# Importing the folium library for creating interactive maps.
import folium

# Selecting specific columns from the dataframe and dropping duplicate entries based on latitude and longitude. Converting the result to a Pandas dataframe.
df_cell_used = df.select(['cell_id','latitude','longitude']).dropDuplicates(['latitude','longitude']).toPandas()

# Converting the df_range dataframe to a Pandas dataframe for further processing.
df_range_pd = df_range.toPandas()

# Defining the coordinates for the corners of a rectangular area.
upper_left = (df_range_pd['min_latitude'].iloc[0], df_range_pd['min_longitude'].iloc[0])
upper_right = (df_range_pd['max_latitude'].iloc[0], df_range_pd['min_longitude'].iloc[0])
lower_left = (df_range_pd['min_latitude'].iloc[0], df_range_pd['max_longitude'].iloc[0])
lower_right = (df_range_pd['max_latitude'].iloc[0], df_range_pd['max_longitude'].iloc[0])

# Storing the corner coordinates in a list representing the edges of the rectangular area.
edges = [upper_left, upper_right, lower_right, lower_left]

# Creating a folium map centered at a specified location with a defined zoom level and using a specific tileset.
cell_location = folium.Map(
    location=[df_range_pd['min_latitude'].iloc[0], df_range_pd['min_longitude'].iloc[0]], # Centering the map at the minimum latitude and longitude coordinates.
    zoom_start=10,  # Setting the initial zoom level of the map.
    tiles="Cartodb Positron"  # Using the "Cartodb Positron" tileset for the map.
)

# Adding a polygon to the map, representing the rectangular area defined by the edges with specified color and fill properties.
folium.Polygon(locations=edges, color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.2).add_to(cell_location)
# folium.Rectangle(bounds=edges, color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.2).add_to(cell_location)

# Iterating through the rows of the df_cell_used dataframe and adding markers to the map for each cell tower location.
for i, row in df_cell_used.sample(100).iterrows():
    folium.Marker(
        location=[row["latitude"], row["longitude"]],  # Specifying the coordinates of the marker based on its latitude and longitude.
        popup=row['cell_id'],  # Displaying the tower ID as the popup message.
        icon=folium.Icon(icon='signal')  # Defining the icon to use for each marker (signal).
    )\
    .add_to(cell_location)  # Adding the marker to the map.

# Displaying the final map with all the added elements (polygon and markers).
cell_location

### Explore Records Distribution

The code below will group a dataframe called `df` by the `'msisdn'` column and computes various count functions for other columns. 

The resulting dataframe `df_user_dist` is created using Apache Spark SQL `groupBy()` method, which groups the data by 'msisdn' column. Then it applies several aggregate functions to each group of data using the `agg()` method. 

In particular, the following five aggregate functions are applied to different columns:
- `count(col('msisdn'))`: counts the number of non-null rows in the 'msisdn' column for each group
- `countDistinct(col('msisdn'))`: counts the number of distinct 'msisdn' values for each group
- `countDistinct(col('cell_id'))`: counts the number of distinct 'cell_id' values for each group
- `countDistinct(col('data_type'))`: counts the number of distinct 'data_type' values for each group
- `countDistinct(col('service'))`: counts the number of distinct 'service' values for each group

The resulting columns are then renamed to more meaningful names using the `alias()` method.

Finally, the result from spark dataframe is collected into pandas dataframe by using the `toPandas()` method. This data will be used for plotting the records distribution to identify anomaly subscribers such as robot or tourist.

The `.head()` method is called on the resulting pandas dataframe `df_user_dist` to display the first 5 rows of data from it.

In [ ]:
# This code groups the dataframe df by 'msisdn' column and applies various count functions for other columns.
df_user_dist = (
    df.groupBy("msisdn")
      .agg(
          f.count("*").alias("records"),
          f.countDistinct("date").alias("unique_date"),
          (f.count("*") / f.countDistinct("date")).alias("avg_records_day"),
          f.countDistinct("cell_id").alias("unique_cell_id"),
          f.countDistinct("data_type").alias("unique_data_type"),
          f.countDistinct("service").alias("unique_service"),
      )
)

# If you really need pandas:
df_user_dist = df_user_dist.toPandas()
df_user_dist = df_user_dist.sort_values("avg_records_day", ascending=False)

To see the distribution of the records per subscribers, we can create a histogram of the 'records' column of df_user_dist data.

When analyzing the distribution of the number of records in Mobile Positioning Data (MPD), it's possible to observe certain patterns that may align with `common anomalies` that often occur. These patterns can provide insights into potential issues or irregularities within the data.

In a healthy dataset, the number of records may exhibit a relatively `uniform distribution` across different subscribers, locations, etc. This indicates a consistent and expected behavior of data collection. If there is a deviation from the uniform distribution, such as a significant difference in the number of records in some particular subscribers compared to the others, it may indicate an issue worth investigating. For example, a high frequency events from a subscribers in a single cell location could be due to the use of robot system to interact with mobile network.

In [ ]:
# Creates a histogram of the 'records' column of df_user_dist data frame with 20 bins
df_user_dist['records'].hist(bins=20)

After exploring the distribution at the user level, we can explore more deeper by checking the distribution at cell_id level for each subscribers, so that we can see more granular pattern and identify the potential anomaly subscribers inside the data, such as robots or tourist.

This code line performs the following tasks:

- It groups `df` by unique combinations of `'msisdn'`, `'date'`, and `'cell_id'` columns using `groupBy()` function.
- After grouping, it computes aggregation functions on each group. Here, it calculates the count of records for each unique combination of the three columns. Additionally, it counts the number of distinct entries in the 'data_type' and 'service' columns to get the number of unique data types and services in each group.
- Finally, it collects the aggregated data from Spark dataframe into a new Pandas dataframe using `toPandas()` function.

In [ ]:
# Groups the df data frame by unique combinations of 'msisdn', 'date', and 'cell_id' columns
df_subs_cell_date = df \
  .groupBy(['msisdn','date','cell_id']) \
  .agg(
      # Counts the total number of records for each unique value of 'msisdn' and renames that column as 'records'
      f.count(col('msisdn')).alias('records'),
      # Counts the number of unique 'data_type' values for each group and renames that column as 'unique_data_type'
      f.countDistinct(col('data_type')).alias('unique_data_type'),
      # Counts the number of unique 'service' values for each group and renames that column as 'unique_service'
      f.countDistinct(col('service')).alias('unique_service')
  ).toPandas() # collect data from spark into pandas

# Sorts the resulting data frame by the 'records' column in descending order
df_subs_cell_date.sort_values('records',ascending=False)


In [ ]:
# Selects the 'records' column from the df_subs_patterns dataframe and plots a histogram with 100 bins
df_subs_cell_date['records'].hist(bins=50)

### Remove Anomaly Subscribers

#### Filtering Robot

Here, `df_subs_stats` is assumed to be a dataframe containing information about subscriber statistics, and `msisdn` is assumed to be one of the columns containing phone numbers. The code determines the subscribers who have made more than 500 (or configured threshold) events in a single site at a single day (which are treated as robots), calculates their percentage, and prints it out along with the total count. Finally, it provides the list of the first 10 msisdns (phone numbers) of those subscribers.

Step by step:
1. Selects a subset of data from a pandas dataframe `df_subs_stats` where the value in the column 'max_records' is greater than 500 and assigns it to a new dataframe called `df_robots`.
2. Calculates the number of rows in the `df_robots` dataframe and assigns it to a variable `n_robots`.
3. Calculates the percentage of subscribers who have more than 500 records in a single site at a single day and assigns it to a variable `prop_robots`.
4. Prints out the detection message of the detected number of subscribers and the corresponding percentage.
5. Shows the first 10 values of the `msisdn` column from the `df_robots` dataframe.


In [ ]:
print("Robot Threshold: {}".format(RAW_CONF['ROBOT_THRESHOLD']))

In [ ]:
# Groups the df_subs_patterns dataframe by 'msisdn' and calculates aggregated statistics for the 'records' column
df_subs_cell_date_stats = df_subs_cell_date.groupby('msisdn').agg(
    min_records=('records','min'),
    med_records=('records','median'),
    max_records=('records','max')
).reset_index()

# Prints out the resulting dataframe
df_subs_cell_date_stats.sort_values('max_records',ascending=False).head(10)

In [ ]:
df_subs_cell_date_stats['max_records'].hist(bins=20)

In [ ]:
# Select all rows from the DataFrame `df_subs_stats` where 'max_records' is greater than the robot threshold and assign it to a new DataFrame called `df_robots`.
df_robots = df_subs_cell_date_stats[df_subs_cell_date_stats['max_records']>RAW_CONF['ROBOT_THRESHOLD']]

# Calculate the number of rows in the `df_robots` DataFrame and store the value in a variable called `n_robots`.
n_robots = len(df_robots)

# Calculate the percentage of robots (subscribers with more than the robot threshold of events in a single site at a single day) and store it in a variable called `prop_robots`.
prop_robots = n_robots/(len(df_subs_cell_date_stats))*100

# Print out detection message which shows the total number of detected robots and their percentage.
print('Detecting {} subscribers ({:.2f}%) with more than {} events in a single site at a single day.'.format(n_robots,prop_robots,RAW_CONF["ROBOT_THRESHOLD"]))

# Convert the first 10 values of the 'msisdn' column from the `df_robots` DataFrame into a list and print it.
df_robots['msisdn'].tolist()[:10]

Filters out any rows from the original `df` DataFrame whose 'msisdn' values match any of the IDs listed in `df_robots`. The filtered DataFrame is assigned to `df_robot_filtered`.

In [ ]:
# Filter out any rows from the DataFrame `df` that have 'msisdn' values that appear in the `anomaly_subscribers` list, and assign the resulting DataFrame to `df_filtered`.
df_robot_filtered = df.filter(~df.msisdn.isin(df_robots['msisdn'].unique().tolist()))

# Show the first 5 rows of the filtered DataFrame.
df_robot_filtered.show(5)

# Print out the count (total number of rows) in the filtered DataFrame.
print("Number of records before robot filtering: {}".format(df.count()))
print("Number of records after robot filtering: {}".format(df_robot_filtered.count()))

### Filter Random Events

This code creates a PySpark DataFrame `df_robot_filtered` and performs the following operations:
* Creates a window specification called `w_subs_site_date` that partitions data by columns `'msisdn'`, `'cell_id'`, and `'date'`.
* Uses the `withColumn()` method to add a new column called `'records_site'` to `df_robot_filtered`, which counts the number of records for each subscriber (`'msisdn'`) at each site (`'cell_id'`) per day (`'date'`). The `count()` function runs over the window specification `w_subs_site_date`.
* Filters the resulting DataFrame `df_robot_notrandom_filtered` to keep only the rows where the value in the `'records_site'` column is greater than 1.

In [ ]:
print("Random Event Threshold: {}".format(RAW_CONF['RANDOM_EVENT_THRESHOLD']))

In [ ]:
# # Define a window object called `w_subs_site_date` that partitions by 'msisdn', 'cell_id', and 'date'
# w_subs_site_date = Window.partitionBy(['msisdn', 'cell_id', 'date'])
# Define a window object called `w_subs_site_date` that partitions by 'msisdn', 'cell_id'
w_subs_site_date = Window.partitionBy(['msisdn', 'cell_id'])

# Create a new DataFrame `df_robot_notrandom_filtered` using `df_robot_filtered`
# Add a new column called 'records_site' using count() function with the window defined above
df_robot_notrandom_filtered = df_robot_filtered \
    .withColumn('records_site', f.count('data_type').over(w_subs_site_date))

# Print DataFrame and the number of records in it before filtering
print("Number of records before random records filtering: {}".format(df_robot_notrandom_filtered.count()))
df_robot_notrandom_filtered.show(10)

# Filter df_robot_notrandom_filtered to keep only rows where 'records_site' is greater than threshold
df_robot_notrandom_filtered = df_robot_notrandom_filtered.filter(col('records_site') > RAW_CONF["RANDOM_EVENT_THRESHOLD"])

# Print DataFrame and the number of records in it after filtering
df_robot_notrandom_filtered.show(10)

print("Number of the records after random records filtering: {}".format(df_robot_notrandom_filtered.count()))

### Filtering Tourists Subcribers - step 1

In here, tourist is defined as subscribers with less than X total events for all sites in a month within our targeted area of home and work location. For example, in a given month, Subscriber "A" only has X-1 total events (visits to different locations) within the targeted area. Since this is less than X, Subscriber "A" meets the criteria for being classified as a tourist.

The code selects all rows from the `df_subs_stats` DataFrame where the value in the 'max_records' column is less than TOURIST_EVENT_THRESHOLD (the configured threshold), indicating that these subscribers have performed less than TOURIST_EVENT_THRESHOLD events across all sites. It creates a new DataFrame called `df_tourists` to store these rows.

It then calculates the number of rows in the `df_tourists` DataFrame (representing the total number of "tourist" subscribers), as well as the percentage of all subscribers that fall into this category.

The exclusion of tourist data from the analysis of usual residence holds significance in ensuring an accurate representation of the local population, thereby enhancing the representativeness of the estimated percentage of internet usage in a specific area.

Note: If tourist 1 filtering should not be done, set the parameters TOURIST_EVENT_THRESHOLD to 0 in the conf file.

In [ ]:
print("Tourist Threshold: {}".format(RAW_CONF['TOURIST_EVENT_THRESHOLD']))

In [ ]:
from pyspark.sql.functions import month

df_robot_notrandom_filtered = df_robot_notrandom_filtered.withColumn('month',month('datetime'))

df_subs_cell_month = df_robot_notrandom_filtered \
  .groupBy(['msisdn','month','cell_id','data_type','service']) \
  .agg(
      # Counts the total number of records for each unique value of 'msisdn' and renames that column as 'records'
      f.count(col('msisdn')).alias('records'),
      # Counts the number of unique 'data_type' values for each group and renames that column as 'unique_data_type'
      f.countDistinct(col('data_type')).alias('unique_data_type'),
      # Counts the number of unique 'service' values for each group and renames that column as 'unique_service'
      f.countDistinct(col('service')).alias('unique_service')
  ).toPandas() # collect data from spark into pandas

# Sorts the resulting data frame by the 'records' column in descending order
df_subs_cell_month.sort_values('records',ascending=False)

In [ ]:
# Example
# df_subs_cell_month[df_subs_cell_month['msisdn']=='subscribers_00005'].sort_values('records',ascending=False)

In [ ]:
# Groups the df_subs_patterns dataframe by 'msisdn' and calculates aggregated statistics for the 'records' column
df_subs_cell_month_stats = df_subs_cell_month.groupby('msisdn').agg(
    min_records=('records','min'),
    med_records=('records','median'),
    max_records=('records','max')
).reset_index()

# Prints out the resulting dataframe
df_subs_cell_month_stats.sort_values('max_records',ascending=False)

In [ ]:
# Filter rows from `df_subs_cell_month_stats` to keep only those where 'max_records' is less than 7. 
df_tourists = df_subs_cell_month_stats[df_subs_cell_month_stats['max_records']<RAW_CONF['TOURIST_EVENT_THRESHOLD']]

# Count the number of rows in `df_tourists`
n_tourists = len(df_tourists)

# Calculate the proportion of 'tourist' subscribers in `df_subs_cell_month_stats`
prop_tourists = n_tourists/(len(df_subs_cell_month_stats))*100

# Display a message about the number and proportion of subscribers identified as tourists,
# using f-string formatting to insert the values of `n_tourists` and `prop_tourists` into the message.
print('Detecting {} subscribers ({:.2f}%) with less than {} events from all sites.'.format(n_tourists,prop_tourists,RAW_CONF["TOURIST_EVENT_THRESHOLD"]))

# Display the first 10 elements in the 'msisdn' column of the DataFrame 'df_tourists'
df_tourists['msisdn'].tolist()[:10]

In [ ]:
# Filter rows from `df_robot_notrandom_filtered` such that they do not contain any of the MSISDNs in `df_tourists`,
# and assign the result to a new DataFrame called `df_all_filtered`.
df_tourist1_filtered = df_robot_notrandom_filtered.filter(~df_robot_notrandom_filtered.msisdn.isin(df_tourists['msisdn'].unique().tolist()))

# Show the first 5 records of the filtered DataFrame.
df_tourist1_filtered.show(5)

# Print the number of rows in the final filtered DataFrame.
print("Number of records after all filtering: {}".format(df_tourist1_filtered.count()))

### Filtering Tourists Subcribers - step 2

This part identifies a second type of tourists. Tourist 2 is defined as subscribers with less than Y active days and less than Z active consecutive days in the entire time period within our targeted area of home and work location. 

For example, in a given month, Subscriber "A" is only active for Y-1 days and Z-1 days in a row within the targeted area. Since this is less than Y and Z, Subscriber "A" meets the criteria for being classified as a tourist 2.

The code calculates the number of total days and the number of days in a row for each subscriber. If both of these are less than Y and Z the subscriber is added to a new DataFrame called `df_tourists_2` to store these rows.

It then calculates the number of rows in the `df_tourists_2` DataFrame (representing the total number of "tourist 2" subscribers), as well as the percentage of all subscribers that fall into this category.

The exclusion of tourist 2 data from the analysis of usual residence holds significance in ensuring an accurate representation of the local population, thereby enhancing the representativeness of the estimated percentage of internet usage in a specific area. 

Note: If tourist 2 filtering should not be done, set the parameters TOURIST_DAYS_THRESHOLD and TOURIST_DAYSROW_THRESHOLD to 0 in the conf file.

In [ ]:
print("Max number of days for tourist activity: {}".format(RAW_CONF['TOURIST_DAYS_THRESHOLD']))
print("Max number of consecutive days for tourist activity: {}".format(RAW_CONF['TOURIST_DAYSROW_THRESHOLD']))

In [ ]:
from pyspark.sql import functions as F

# ------------------------------------------------------------
# 1. Get one row per msisdn per active date
# ------------------------------------------------------------
df_days = df.select("msisdn", "date").distinct()

# ------------------------------------------------------------
# 2. Compute total number of active days per msisdn
# ------------------------------------------------------------
df_total_days = df_days.groupBy("msisdn") \
    .agg(F.count("*").alias("total_active_days"))

# ------------------------------------------------------------
# 3. Compute longest streak of consecutive active days
# ------------------------------------------------------------
# 3a. Add day_number, row_number and group key
w = Window.partitionBy("msisdn").orderBy("date")

df_days2 = df_days \
    .withColumn("day_number", F.datediff(col("date"), F.lit("1970-01-01"))) \
    .withColumn("rn", F.row_number().over(w)) \
    .withColumn("grp", col("day_number") - col("rn"))

# 3b. Count the size of each consecutive-day group
df_streaks = df_days2.groupBy("msisdn", "grp") \
    .agg(F.count("*").alias("streak_len"))

# 3c. Extract the longest consecutive streak per msisdn
df_longest_streak = df_streaks.groupBy("msisdn") \
    .agg(F.max("streak_len").alias("max_consecutive_days"))

# ------------------------------------------------------------
# 4. Combine active days + longest streak
# ------------------------------------------------------------
df_activity_stats = df_total_days \
    .join(df_longest_streak, "msisdn", "inner")

# ------------------------------------------------------------
# 5. Select tourists: < 7 total days AND < 7 consecutive days
# ------------------------------------------------------------
df_tourists_2 = df_activity_stats \
    .filter(
        (col("total_active_days") < RAW_CONF['TOURIST_DAYS_THRESHOLD']) &
        (col("max_consecutive_days") < RAW_CONF['TOURIST_DAYSROW_THRESHOLD'])
    ).toPandas()

In [ ]:
# Count the number of rows in `df_tourists_2`
n_tourists_2 = len(df_tourists_2)

# Calculate the proportion of 'tourist 2' subscribers in `df_subs_cell_month_stats`
prop_tourists_2 = n_tourists_2/(len(df_subs_cell_month_stats))*100

# Display a message about the number and proportion of subscribers identified as tourists 2,
# using f-string formatting to insert the values of `n_tourists_2` and `prop_tourists_2` into the message.
print('Detecting {} subscribers ({:.2f}%) with less than {} active days and max {} consequtive days from all sites.'.format(n_tourists_2,prop_tourists_2,RAW_CONF["TOURIST_DAYS_THRESHOLD"],RAW_CONF["TOURIST_DAYSROW_THRESHOLD"]))

# Display the first 10 elements in the 'msisdn' column of the DataFrame 'df_tourists'
df_tourists_2['msisdn'].tolist()[:10]

In [ ]:
# Filter rows from `df_tourist1_filtered` such that they do not contain any of the MSISDNs in `df_tourists_2`,
# and assign the result to a new DataFrame called `df_all_filtered`.
df_all_filtered = df_tourist1_filtered.filter(~df_tourist1_filtered.msisdn.isin(df_tourists_2['msisdn'].unique().tolist()))

# Show the first 5 records of the filtered DataFrame.
df_all_filtered.show(5)

# Print the number of rows in the final filtered DataFrame.
print("Number of records after all filtering: {}".format(df_all_filtered.count()))

### Saving data for the next step

Finally, after finishing all the filtering process, we can save the filtered data to be used for the next steps. 

The data is partitioned by the column 'msisdn'. Partitioning is the process of dividing data into multiple smaller and manageable parts based on some criteria or column(s). In this case, partitioning is done on 'msisdn', which means that each unique value of 'msisdn' will be stored in a separate subdirectory in the output folder. Partitioning often helps with faster processing of large datasets by reading/writing only necessary partition(s) instead of the whole dataset.

The output data will be written to the directory specified by `'BASE_PATH'`, with the subdirectory name `'MPD_sample_synthetic_filtered'`. The exact path to the output directory depends on the value of `'BASE_PATH'`. 

In [ ]:
df_all_filtered.select(
      f.countDistinct(col('msisdn')).alias('unique_subscribers'), # Counting distinct values of 'msisdn' column and giving it an alias name 'unique_subscribers'
      f.countDistinct(col('cell_id')).alias('unique_cell_id'), # Counting distinct values of 'cell_id' column and giving it an alias name 'unique_cell_id'
      f.countDistinct(col('data_type')).alias('unique_data_type'), # Counting distinct values of 'data_type' column and giving it an alias name 'unique_data_type'
      f.countDistinct(col('service')).alias('unique_service') # Counting distinct values of 'service' column and giving it an alias name 'unique_service'
  ).show() # Displaying the resulting dataframe using show() method

In [ ]:
# Write the DataFrame `df_all_filtered` to disk in the Parquet file format using the `write` method.
# Before write the file, we can drop the records_site and month column since it will not be used in further analysis
# The data will be partitioned by the column 'msisdn' using the `partitionBy()` method.
# This means that each unique value of 'msisdn' will be stored in a separate subdirectory in the output folder.
# The output data will be written to the directory specified by 'BASE_PATH', with the subdirectory name 'MPD_sample_synthetic_filtered'.
df_all_filtered = df_all_filtered.drop('records_site').drop('month')

df_all_filtered\
    .write.mode("overwrite")\
    .partitionBy('msisdn')\
    .parquet(BASE_PATH+FILTERED_FILE_PATH_PARQUET)

In [ ]:
df_all_filtered = df_all_filtered.drop('records_site').drop('month')

df_all_filtered\
    .coalesce(1)\
    .write.option("header",True)\
    .mode("overwrite")\
    .csv(BASE_PATH+FILTERED_FILE_PATH_CSV)

In [ ]:
spark.stop()